In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
import dotenv

In [3]:
dotenv.load_dotenv()

True

In [4]:
from langchain_deepseek import ChatDeepSeek
import json
from typing import Annotated
from src.utils_stock import get_top_nasdaq_stock_data
from langchain_core.tools import tool
from langgraph.prebuilt import InjectedState
from langgraph.prebuilt import ToolNode
from typing import TypedDict
from langgraph.graph.message import add_messages

In [5]:
llm = ChatDeepSeek(model="deepseek-chat",
                   api_key=os.getenv("DEEPSEEK_API_KEY"))

In [8]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    number_days_of_analysis: int
    email: str

### data_acquisition_tool_node

In [6]:
@tool
def get_top_nasdaq_performance_stock_data(
        state: Annotated[dict, InjectedState]) -> str:
    """Get the day's top NASDAQ-100 gainer with symbol, percentage increase, sample data in markdown format, pandas dataframe info and csv data path for further python code analysis.
    """
    print(state.keys())
    result = get_top_nasdaq_stock_data(
        state["number_days_of_analysis"])
    return json.dumps(result)

In [7]:
data_acquisition_tools = [get_top_nasdaq_performance_stock_data]
data_acquisition_tool_node = ToolNode(data_acquisition_tools)

### 

### data_acquisition_agent_node

In [10]:
def data_acquisition_agent_node(state: State):
    messages = state["messages"]
    llm_with_tools = llm.bind_tools(data_acquisition_tools)
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

In [38]:
from typing import Literal

from langgraph.graph import StateGraph


workflow = StateGraph(State)

workflow.add_node("agents", data_acquisition_agent_node)
workflow.add_node("tool", data_acquisition_tool_node)


def should_continue(state: State) -> Literal["tool", "__end__"]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tool"
    return "__end__"


workflow.add_edge("__start__", "agents")
workflow.add_conditional_edges(
    "agents",
    should_continue,
)
workflow.add_edge("tool", "agents")

app = workflow.compile()

In [39]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [10]:

from langchain_core.messages import HumanMessage

for chunk in app.stream(
    input={
        "messages": [
        HumanMessage(
            content="Let get the top performance stock data")
        ],
        "number_days_of_analysis": 5,
        "email": "nelsonlin0321@gmail.com"
    },
    stream_mode="values",
    # config={"configurable": {"number_days_of_analysis": "123"}}
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Let get the top performance stock data
================================== Ai Message ==================================
Tool Calls:
  get_top_nasdaq_performance_stock_data (call_0_98f7fb33-02c9-4d2b-b0be-cae6c3ac881e)
 Call ID: call_0_98f7fb33-02c9-4d2b-b0be-cae6c3ac881e
  Args:
dict_keys(['messages', 'number_days_of_analysis', 'email'])


100%|██████████| 101/101 [00:02<00:00, 38.62it/s]


================================= Tool Message =================================
Name: get_top_nasdaq_performance_stock_data

{"symbol": "MNST", "percentage_increased": 3.7238540649033007, "sample_data_in_markdown": "| Date                      |   Open |   High |   Low |   Close |   Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|------:|--------:|---------:|------------:|---------------:|\n| 2025-05-05 00:00:00-04:00 |  60.06 |  61.29 | 59.63 |   60.91 |  4609300 |           0 |              0 |\n| 2025-05-06 00:00:00-04:00 |  60.58 |  60.77 | 59.89 |   60.02 |  5258100 |           0 |              0 |\n| 2025-05-07 00:00:00-04:00 |  60.26 |  60.85 | 59.98 |   60.56 |  3870900 |           0 |              0 |\n| 2025-05-08 00:00:00-04:00 |  60.65 |  61.07 | 60.08 |   60.14 |  6083900 |           0 |              0 |\n| 2025-05-09 00:00:00-04:00 |  58.81 |  61.83 | 58.62 |   61    |  8654400 |           0 |              0 |", "data_csv_path": "/